In [1]:
# Lab-05 Logistic Regression -> binary classification

import torch
import torch.nn as nn
import torch.nn.functional as F # -> torch.nn.functional에서 제공하는 Loss ftn. 사용
import torch.optim as optim

# For reproducibility
torch.manual_seed(1) # random seed를 줌 / 실험할때 마다 다른 값들의 연산으로 결과값이 달라지는 걸 방지하고자 동일한 난수의 사용이 필요함

/Users/paul/opt/miniconda3/envs/qtc/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [58]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [59]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

e^1 equals:  tensor([2.7183])


In [60]:
hypothesis = 1/(1+torch.exp(-(x_train.matmul(W)+b)))

In [61]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [62]:
print('1/(1+e^{-1} equals: )', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1} equals: ) tensor([0.7311])


In [63]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)

In [64]:
print(hypothesis)
print(hypothesis.shape)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [65]:
-(y_train[0] * torch.log(hypothesis[0]) + (1 - y_train[0]) * torch.log(1 - hypothesis[0])) 

tensor([0.6931], grad_fn=<NegBackward0>)

In [66]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [67]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [68]:
F.binary_cross_entropy(hypothesis, y_train) # 위에 losses에서 쓴 식을 담고 있는 모듈임. bce라고도 함. 

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

In [69]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    # Cost 계산 
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 그전에 혹시 gradinent 계산한게 있으면 초기화하는 것 -> 기존에 구해둔게 있다면 계속 더하게 되기 때문.
    cost.backward() # back propagation을 함
    optimizer.step() # cost값을 minimize하는 방향으로 W, b를 구함

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [70]:
# training set에 대해 훈련했으니까 훈련평가를 해야함
hypothesis = torch.sigmoid(x_train.matmul(W)+b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward0>)


In [71]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [72]:
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [73]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [6]:
class BinaryClassfier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [7]:
model = BinaryClassfier()

In [8]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        predictino = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = predictino.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost {:.6f} Accuracy {:2.2f}%'.format(epoch, nb_epochs, cost.item(), accuracy*100,))

Epoch    0/100 Cost 0.734527 Accuracy 50.00%
Epoch   10/100 Cost 0.446570 Accuracy 66.67%
Epoch   20/100 Cost 0.448868 Accuracy 66.67%
Epoch   30/100 Cost 0.375859 Accuracy 83.33%
Epoch   40/100 Cost 0.318583 Accuracy 83.33%
Epoch   50/100 Cost 0.268096 Accuracy 83.33%
Epoch   60/100 Cost 0.222295 Accuracy 100.00%
Epoch   70/100 Cost 0.183465 Accuracy 100.00%
Epoch   80/100 Cost 0.158036 Accuracy 100.00%
Epoch   90/100 Cost 0.144541 Accuracy 100.00%
Epoch  100/100 Cost 0.134652 Accuracy 100.00%
